In [1]:
# The purpose of this part is to upload training file
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

# Since we are using google colab so the above commands are just to upload the file to google drive for further processing
# You can skip this part if you are are using your own local GPU/CPU

Saving ISEAR.csv to ISEAR.csv
User uploaded file "ISEAR.csv" with length 914779 bytes


In [2]:
# Importing all the required NLTK libraries for further usage
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /content/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /content/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /content/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /content/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /content/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /content/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /c

       |   Unzipping corpora/omw.zip.
       | Downloading package opinion_lexicon to /content/nltk_data...
       |   Unzipping corpora/opinion_lexicon.zip.
       | Downloading package paradigms to /content/nltk_data...
       |   Unzipping corpora/paradigms.zip.
       | Downloading package pil to /content/nltk_data...
       |   Unzipping corpora/pil.zip.
       | Downloading package pl196x to /content/nltk_data...
       |   Unzipping corpora/pl196x.zip.
       | Downloading package ppattach to /content/nltk_data...
       |   Unzipping corpora/ppattach.zip.
       | Downloading package problem_reports to /content/nltk_data...
       |   Unzipping corpora/problem_reports.zip.
       | Downloading package propbank to /content/nltk_data...
       | Downloading package ptb to /content/nltk_data...
       |   Unzipping corpora/ptb.zip.
       | Downloading package product_reviews_1 to
       |     /content/nltk_data...
       |   Unzipping corpora/product_reviews_1.zip.
       | Downl

       |   Unzipping tokenizers/punkt.zip.
       | Downloading package book_grammars to /content/nltk_data...
       |   Unzipping grammars/book_grammars.zip.
       | Downloading package sample_grammars to /content/nltk_data...
       |   Unzipping grammars/sample_grammars.zip.
       | Downloading package spanish_grammars to /content/nltk_data...
       |   Unzipping grammars/spanish_grammars.zip.
       | Downloading package basque_grammars to /content/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package large_grammars to /content/nltk_data...
       |   Unzipping grammars/large_grammars.zip.
       | Downloading package tagsets to /content/nltk_data...
       |   Unzipping help/tagsets.zip.
       | Downloading package snowball_data to /content/nltk_data...
       | Downloading package bllip_wsj_no_aux to /content/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package word2vec_sample to /content/nltk_data

True

# Data Set: We are using ISEAR Dataset (An open source) for emotion detection


The ISEAR dataset has been developped by the Swiss National Center of Competence in Research.

The dataset contains 7 different emotions namely "Anger", "Disgust", "Fear", "Guilt", "Joy", "Sadness", "Shame". The emotions are taged based on the content present within a sentence. Each sentence is tagged with either one of the 7 emotions. There are 7570 sentences present within dataset.

The ISEAR dataset is balanced as apporximately all of the emotions are repeating same number of times within the data.

The purpose of this module is to import the csv data and store it into the list format for processing.

In [3]:
# Importing the required libraries
from nltk.corpus import stopwords
import pandas as pd
from sklearn.cross_validation import train_test_split # This library is used for splitting the data for training and testing
import io
import string
from nltk.stem.wordnet import WordNetLemmatizer
df = pd.read_csv('ISEAR.csv') # Reading the dataframe

a = pd.Series(df['joy'])
b = pd.Series(df[' On days when I feel close to my partner and other friends  When I feel at peace with myself and also experience a close contact with people whom I regard greatly'])

new_df = pd.DataFrame({'Text': b, 'Emotion': a})

em_list = []
text_list = []

## create the training set
train = []
xtrain=[]
xtest=[]
exclude = set(string.punctuation)  ## stores all the punctuations
lemma = WordNetLemmatizer() # used for lemmatization
stop = set(stopwords.words('english'))
negative=['']

# The clean function is used to remove stop words, punctuations, perform lemmatization and remove any negative words if desired
def clean(doc):
  stop_free = " ".join([i for i in doc.lower().split() if i not in stop if i not in negative])
  punc_free = "".join([ch for ch in stop_free if ch not in exclude])
  normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
  without_digits = " ".join([x for x in normalized.split() if not (x.isdigit() 
                                          or x[0] == '-' and x[1:].isdigit())])
  return without_digits
# Defining all the categories
all_categories = ['anger','disgust','fear','guilt','joy','sadness','shame']

for i in range(df.shape[0]):
  new_df.loc[i]['Text'] = clean(new_df.loc[i]['Text'])

for i in range(new_df.shape[0]):
  text_list.append(new_df.loc[i]['Text'])
  em_list.append(new_df.loc[i]['Emotion'])
  train.append([text_list[i], em_list[i]])
            #print(self.train[0][1])

xtrain, xtest = train_test_split(train, test_size=0.2) # Splitting the data set into 80% Training and 20% Testing


/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# One Hot Encoding

All of the unique words present within each sentence is encoded using OneHotEncoder

In [0]:
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas  
import numpy as np

grandpair=[]

# This for loop is used to get unique words within the data set for lavel encoding and then one hot encoding respectively
for i in range(0,len(train)):
  pairs= train[i][0].split()
  grandpair = grandpair + pairs
  
unique_pair = list(set(grandpair)) 

label_encoder = LabelEncoder()
label_encoder.fit(unique_pair)
feature = label_encoder.transform(unique_pair)


onehot_encoder = OneHotEncoder(sparse=False,n_values=len(unique_pair))

# Now we are summing up all the one hot encoding for each sentence so that we can incorporate variable length sentence...
# The shape of each row is approx 74700 columns which is the size of dictionary 

xdata_onehot = np.zeros((len(train),len(unique_pair)))
xdata_encoded = np.zeros((len(train),100))

for j in range (0,len(train)):
  sentence = label_encoder.transform(train[j][0].split())
  for k in range(0,len(sentence)):
    xdata_encoded[j][k]= sentence[k]
  sentence = sentence.reshape(len(sentence), 1)
  encoded_features = onehot_encoder.fit_transform(sentence)

  xdata_onehot_temp= np.zeros((1,len(unique_pair)))
  for k in range (0,len(encoded_features)):
    xdata_onehot_temp = np.add(xdata_onehot_temp,encoded_features[k,:])
  xdata_onehot[j]= xdata_onehot_temp


# Now all the emotions are also one hot encoded  
emotion_encoder1 = LabelEncoder()
emotionpair=[]
number_of_emotions=7
for i in range(0,len(train)):
  pairs= train[i][1].split()
  emotionpair = emotionpair + pairs 
  
emotion_encoder1.fit(emotionpair)
emotion_label = np.zeros((len(train),1))

onehot_emotion_enc = OneHotEncoder(sparse=False,n_values=7)


for j in range (0,len(train)):
  sentence = emotion_encoder1.transform(train[j][1].split())
  sentence = sentence.reshape(len(sentence), 1)
  emotion_label[j]= sentence



Previously, we are performing one hot encoding on all of the data set. Now, we are seprately encoding the training and test data set for further analyzing our model.

In [0]:
# Train and test data is separately encoded
traindata_encoded = np.zeros((len(xtrain),100))
testdata_encoded = np.zeros((len(xtest),100))
trainlabel_encoded = np.zeros((len(xtrain),1))
testlabel_encoded = np.zeros((len(xtest),1))

for j in range (0,len(xtrain)):
  sentence = label_encoder.transform(xtrain[j][0].split())
  for k in range(0,len(sentence)):
    traindata_encoded[j][k]= sentence[k]
    
for j in range (0,len(xtest)):
  sentence = label_encoder.transform(xtest[j][0].split())
  for k in range(0,len(sentence)):
    testdata_encoded[j][k]= sentence[k]

    
for j in range (0,len(xtrain)):
  sentence = emotion_encoder1.transform(xtrain[j][1].split())
  sentence = sentence.reshape(len(sentence), 1)
  trainlabel_encoded[j]= sentence
  
  
for j in range (0,len(xtest)):
  sentence = emotion_encoder1.transform(xtest[j][1].split())
  sentence = sentence.reshape(len(sentence), 1)
  testlabel_encoded[j]= sentence

# Model Training

## LSTM Cells - Keras

Learning Rate: 0.006, 
Optimizer: Adam, 
Layers: 1 LSTM cell with 140 neurons, 
Dropout: 0.2

In [6]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical
from keras import optimizers

train_labels = to_categorical(trainlabel_encoded, num_classes=None)
test_labels = to_categorical(testlabel_encoded, num_classes=None)
numpy.random.seed(7)
learning_rate=0.006 # Defining the learning rate
epochs=30 # Defining the number of epochs for training
decayrate= learning_rate/epochs
sgd1= optimizers.Adam(lr=learning_rate, beta_1=0.65, beta_2=0.75, epsilon=1e-8, decay=0.07, amsgrad=False) # Adam optimizer is used for model training
# # create the model
embedding_vecor_length = 140
model = Sequential()
model.add(Embedding(len(unique_pair), embedding_vecor_length, input_length=None))
#model.add(LSTM(15,recurrent_activation='relu',unroll=True,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(140,recurrent_activation='relu',unroll=False))
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=sgd1, metrics=['accuracy'])
#print(model.summary())
model.fit(traindata_encoded, train_labels,validation_data=(testdata_encoded, test_labels), nb_epoch=epochs, batch_size=100)
# # Final evaluation of the model
scores = model.evaluate(testdata_encoded, test_labels, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 6056 samples, validate on 1514 samples
Epoch 1/30
6056/6056 [==============================] - 14s 2ms/step - loss: 1.8635 - acc: 0.2498 - val_loss: 1.7789 - val_acc: 0.3236
Epoch 2/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.6976 - acc: 0.3542 - val_loss: 1.7391 - val_acc: 0.3560
Epoch 3/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.6185 - acc: 0.3932 - val_loss: 1.7019 - val_acc: 0.3454
Epoch 4/30
2300/6056 [==========>...................] - ETA: 7s - loss: 1.6098 - acc: 0.3778

6056/6056 [==============================] - 12s 2ms/step - loss: 1.5701 - acc: 0.4072 - val_loss: 1.6618 - val_acc: 0.3884
Epoch 5/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.5208 - acc: 0.4346 - val_loss: 1.6247 - val_acc: 0.3937
Epoch 6/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.4726 - acc: 0.4543 - val_loss: 1.6223 - val_acc: 0.3950
Epoch 7/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.4463 - acc: 0.4647 - val_loss: 1.5924 - val_acc: 0.4135
Epoch 8/30
4800/6056 [======================>.......] - ETA: 2s - loss: 1.4081 - acc: 0.4798

6056/6056 [==============================] - 12s 2ms/step - loss: 1.4095 - acc: 0.4818 - val_loss: 1.5826 - val_acc: 0.4188
Epoch 9/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.3815 - acc: 0.5018 - val_loss: 1.5890 - val_acc: 0.4227
Epoch 10/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.3617 - acc: 0.5040 - val_loss: 1.5691 - val_acc: 0.4201
Epoch 11/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.3382 - acc: 0.5175 - val_loss: 1.5479 - val_acc: 0.4267
Epoch 12/30
5400/6056 [=========================>....] - ETA: 1s - loss: 1.3237 - acc: 0.5150

6056/6056 [==============================] - 12s 2ms/step - loss: 1.3219 - acc: 0.5162 - val_loss: 1.5546 - val_acc: 0.4293
Epoch 13/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.3009 - acc: 0.5246 - val_loss: 1.5547 - val_acc: 0.4313
Epoch 14/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.2905 - acc: 0.5325 - val_loss: 1.5368 - val_acc: 0.4346
Epoch 15/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.2703 - acc: 0.5343 - val_loss: 1.5356 - val_acc: 0.4445
Epoch 16/30
5200/6056 [========================>.....] - ETA: 1s - loss: 1.2554 - acc: 0.5379

6056/6056 [==============================] - 12s 2ms/step - loss: 1.2541 - acc: 0.5401 - val_loss: 1.5219 - val_acc: 0.4399
Epoch 17/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.2308 - acc: 0.5560 - val_loss: 1.5215 - val_acc: 0.4538
Epoch 18/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.2371 - acc: 0.5507 - val_loss: 1.5148 - val_acc: 0.4538
Epoch 19/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.2120 - acc: 0.5575 - val_loss: 1.5029 - val_acc: 0.4531
Epoch 20/30
5400/6056 [=========================>....] - ETA: 1s - loss: 1.2050 - acc: 0.5644

6056/6056 [==============================] - 12s 2ms/step - loss: 1.2058 - acc: 0.5662 - val_loss: 1.4996 - val_acc: 0.4551
Epoch 21/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.2009 - acc: 0.5657 - val_loss: 1.5049 - val_acc: 0.4544
Epoch 22/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.1878 - acc: 0.5694 - val_loss: 1.4982 - val_acc: 0.4597
Epoch 23/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.1875 - acc: 0.5687 - val_loss: 1.4816 - val_acc: 0.4630
Epoch 24/30
5500/6056 [==========================>...] - ETA: 1s - loss: 1.1793 - acc: 0.5742

6056/6056 [==============================] - 12s 2ms/step - loss: 1.1787 - acc: 0.5738 - val_loss: 1.4873 - val_acc: 0.4584
Epoch 25/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.1651 - acc: 0.5814 - val_loss: 1.4945 - val_acc: 0.4696
Epoch 26/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.1562 - acc: 0.5814 - val_loss: 1.4844 - val_acc: 0.4716
Epoch 27/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.1485 - acc: 0.5784 - val_loss: 1.4871 - val_acc: 0.4696
Epoch 28/30
4900/6056 [=======================>......] - ETA: 2s - loss: 1.1464 - acc: 0.5869

6056/6056 [==============================] - 13s 2ms/step - loss: 1.1412 - acc: 0.5888 - val_loss: 1.4833 - val_acc: 0.4749
Epoch 29/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.1424 - acc: 0.5845 - val_loss: 1.4828 - val_acc: 0.4828
Epoch 30/30
6056/6056 [==============================] - 12s 2ms/step - loss: 1.1456 - acc: 0.5859 - val_loss: 1.4735 - val_acc: 0.4782
Accuracy: 47.82%


# Further Analysis of Tranined Model

Since 7 emotions are present in the dataset so we are curious to understand that which emotions are more correctly classified as compared to others. So, we further break down our accuracy into 7 different emotions accuracy

In [8]:
# Initializing true positive and true negative to calculate the accuracy of our model
true_anger=0
true_disgust=0
true_fear=0
true_guilt=0
true_joy=0
true_sadness=0
true_shame=0
false_anger = 0
false_disgust = 0
false_fear = 0
false_guilt = 0
false_joy = 0
false_sadness = 0
false_shame = 0

for i in range(len(testdata_encoded)):
  matrix = np.matrix(testdata_encoded[i])
  matrix1 = np.matrix(testlabel_encoded[i])
  result = model.predict(matrix, batch_size=None, verbose=0, steps=None)
  if matrix1 == [0]:
    if numpy.argmax(result)==matrix1:
        true_anger = true_anger + 1
    else:
        false_anger = false_anger+1

  if matrix1 == [1]:
    if numpy.argmax(result)==matrix1:
        true_disgust = true_disgust + 1
    else:
        false_disgust = false_disgust+1

  if matrix1 == [2]:
    if numpy.argmax(result)==matrix1:
        true_fear = true_fear + 1
    else:
        false_fear = false_fear+1

  if matrix1 == [3]:
    if numpy.argmax(result)==matrix1:
        true_guilt = true_guilt + 1
    else:
        false_guilt = false_guilt+1

  if matrix1 == [4]:
    if numpy.argmax(result)==matrix1:
        true_joy = true_joy + 1
    else:
        false_joy = false_joy+1

  if matrix1 == [5]:
    if numpy.argmax(result)==matrix1:
        true_sadness = true_sadness + 1
    else:
        false_sadness = false_sadness+1

  if matrix1 == [6]:
    if numpy.argmax(result)==matrix1:
        true_shame = true_shame + 1
    else:
        false_shame = false_shame+1

avg_anger = (true_anger / (true_anger+false_anger))
avg_disgust = (true_disgust / (true_disgust + false_disgust))
avg_fear = (true_fear / (true_fear + false_fear))
avg_guilt = (true_guilt / (true_guilt + false_guilt))
avg_joy = (true_joy / (true_joy + false_joy))
avg_sadness =  (true_sadness / (true_sadness + false_sadness))
avg_shame = (true_shame / (true_shame + false_shame))

print("% Anger correct classify:",avg_anger * 100)
print("% Disgust correct classify:",avg_disgust  * 100)
print("% Fear correct classify:",avg_fear  * 100)
print("% Guilt correct classify:",avg_guilt  * 100)
print("% Joy correct classify:",avg_joy  * 100)
print("% Sadness correct classify:",avg_sadness  * 100)
print("% Shame correct classify:",avg_shame  * 100)





% Anger correct classify: 45.851528384279476
% Disgust correct classify: 54.82233502538071
% Fear correct classify: 57.72727272727273
% Guilt correct classify: 33.0188679245283
% Joy correct classify: 62.616822429906534
% Sadness correct classify: 45.410628019323674
% Shame correct classify: 36.59574468085106


#Conclusion

We have used Deep Neural Network to classify emotions present within a sentence. It is further analyzed that not all of the emotions can be predicted accurately. Few of the emotions are easy to predict like "Joy" while other emotions like "Shame" are difficult to predict.


# Future Work
We are further expanding our work to improve the accuracy of our model using other modern tools like 'Attention'.